In [ ]:
import pydicom
import numpy as np

def load_dicom_hu(dicom_path):
    """Convert DICOM to Hounsfield Units"""
    dicom = pydicom.dcmread(dicom_path)
    hu = dicom.pixel_array * dicom.RescaleSlope + dicom.RescaleIntercept
    return hu, dicom

def create_mip(hu_volume, slab_thickness=3):
    """Maximum Intensity Projection for non-gated CTs"""
    return np.max(hu_volume[i:i+slab_thickness] for i in range(len(hu_volume)-slab_thickness))

In [ ]:
import torch
from monai.networks.nets import UNet

def train_unet(ct_volumes, masks):
    """Train cardiac segmentation model"""
    model = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=4,  # Background + 3 arteries
        channels=(16, 32, 64),
        strides=(2, 2)
    )
    optimizer = torch.optim.Adam(model.parameters())
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(100):
        outputs = model(ct_volumes)
        loss = loss_fn(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return model

def extract_roi(hu_volume, model):
    """Predict cardiac ROI using trained U-Net"""
    with torch.no_grad():
        logits = model(torch.tensor(hu_volume[np.newaxis, np.newaxis]))
        mask = torch.argmax(logits, dim=1)
    return hu_volume * (mask.squeeze().numpy() > 0)

In [ ]:
from skimage.measure import label, regionprops

def segment_calcium(hu_volume, hu_min=130, hu_max=3000):
    """Thresholding + connected components"""
    mask = (hu_volume > hu_min) & (hu_volume < hu_max)
    labels = label(mask)
    regions = [r for r in regionprops(labels) if r.area >= 3]

    clean_mask = np.zeros_like(mask)
    for r in regions:
        clean_mask[r.coords[:,0], r.coords[:,1], r.coords[:,2]] = 1
    return clean_mask

In [ ]:
import torch
import torch.nn as nn

class CAD3DCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*8*8*8, 128)  # Adjust based on input size
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        return torch.sigmoid(self.fc2(torch.relu(self.fc1(x))))

def train_model(ct_volumes, labels):
    model = CAD3DCNN()
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(50):
        outputs = model(ct_volumes)
        loss = criterion(outputs, labels.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return model

In [ ]:
def calculate_agatston(calcium_mask, hu_volume, pixel_spacing):
    """Compute Agatston score"""
    labeled = label(calcium_mask)
    score = 0
    for r in regionprops(labeled, hu_volume):
        area = r.area * pixel_spacing[0] * pixel_spacing[1]
        density = min(4, max(1, int(r.mean_intensity//100)))
        score += area * density * max(1, pixel_spacing[2]/3)
    return score

def predict_risk(model, ct_volume, clinical_data):
    """End-to-end prediction"""
    with torch.no_grad():
        risk = model(ct_volume.unsqueeze(0).item()
    return {
        "risk_score": risk,
        "risk_category": "High" if risk > 0.5 else "Low",
        "recommendations": get_recommendations(risk)
    }

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_calcium(hu_slice, calcium_mask):
    plt.imshow(hu_slice, cmap='gray')
    plt.imshow(calcium_mask, cmap='Reds', alpha=0.3)
    plt.title("Calcium Detection")
    plt.show()

def plot_shap(shap_values, features):
    sns.barplot(x=shap_values, y=features)
    plt.title("Feature Importance")
    plt.show()